In [1]:
import math
import numpy as np
import matplotlib.pyplot as plt
import scipy.integrate as spi
import sympy as sp

In [2]:
# 参数设置
S0 = 1      # 初始资产价格
T = 40.0    # 总时长（例如1年）
r = 0.06    # 风险无关利率
sigma = 0.2 # 波动率
dt = 0.01   # 时间步长
N = 500     # 总步数
g = 0
lamda = 1/35

# 定义股价模拟函数
def stock_SDE(S0, r, sigma, dt, N, l):
    S = np.zeros(N)
    S[0] = S0
    for t in range(1, N):
        Z = np.random.normal(0, 1)
        S[t] = S[t-1] * np.exp((r - l - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * Z)
    return S[N-1]

# 定义期权收益函数
def option_payoff(g, t, S0, r, sigma, dt, N, l):
    return max(np.exp(g * t) - stock_SDE(S0, r, sigma, dt, N, l), 0)

# 定义单次积分函数
def single_V(t, l):
    return lamda * np.exp(-lamda * t) * np.exp(-r * t) * option_payoff(g, t, S0, r, sigma, dt, N, l)

# 定义多次积分值的均值计算函数
def average_int_death_payoff(l, M):
    results = []
    for _ in range(M):
        result, _ = spi.quad(single_V, 0, T, args=(l,))
        results.append(result)
    print("积分均值：",np.mean(results))
    return np.mean(results)

# 定义费用流函数
def fee_flows(l):
    return l / (lamda + l)

In [3]:
M = 100  # 例如，进行100次积分
optimal_l = None
min_diff = float('inf')
j = 0
for l_candidate in np.linspace(0, 0.001, 20):  # 尝试不同的 l 值
    average_payoff = average_int_death_payoff(l_candidate, M)
    fee_flow = fee_flows(l_candidate)
    print("回合",j,", fee_flows:", fee_flow)
    diff = abs(average_payoff - fee_flow)
    if diff < min_diff:
        min_diff = diff
        print("回合",j,", diff:",diff)
        optimal_l = l_candidate
        print("回合",j,", optimal_l:",optimal_l)
    j  = j+1
print("Optimal l:", optimal_l)

/tmp/ipykernel_7402/1270861124.py:32: IntegrationWarning: The integral is probably divergent, or slowly convergent.
  result, _ = spi.quad(single_V, 0, T, args=(l,))
/tmp/ipykernel_7402/1270861124.py:32: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  result, _ = spi.quad(single_V, 0, T, args=(l,))


积分均值： 0.023135210084276575
回合 0 , fee_flows: 0.0
回合 0 , diff: 0.023135210084276575
回合 0 , optimal_l: 0.0
积分均值： 0.023140346755106612
回合 1 , fee_flows: 0.0018387181507748885
回合 1 , diff: 0.021301628604331723
回合 1 , optimal_l: 5.2631578947368424e-05
积分均值： 0.02389486751634286
回合 2 , fee_flows: 0.003670686942842161
回合 2 , diff: 0.0202241805735007
回合 2 , optimal_l: 0.00010526315789473685
积分均值： 0.02298785065603366
回合 3 , fee_flows: 0.005495943470295734
回合 3 , diff: 0.017491907185737926
回合 3 , optimal_l: 0.00015789473684210527
积分均值： 0.023980158510649666
回合 4 , fee_flows: 0.0073145245559038665
回合 4 , diff: 0.0166656339547458
回合 4 , optimal_l: 0.0002105263157894737
积分均值： 0.02343307303010813
回合 5 , fee_flows: 0.009126466753585397
回合 5 , diff: 0.014306606276522733
回合 5 , optimal_l: 0.0002631578947368421
积分均值： 0.023768761754555917
回合 6 , fee_flows: 0.010931806350858927
回合 6 , diff: 0.01283695540369699
回合 6 , optimal_l: 0.00031578947368421053
积分均值： 0.02352492375607695
回合 7 , fee_flows: 0.01273057937